# DeepRacer Complete Pipeline Example

## 1. Imports

In [1]:
%reload_ext dotenv
%dotenv

import warnings
warnings.filterwarnings(action='ignore', module='.*paramiko.*')

import os
from pprint import pprint

from src.training_pipeline import train_pipeline, stop_training_pipeline
from src.evaluation_pipeline import evaluate_pipeline, stop_evaluation_pipeline
from src.loganalysis_pipeline import start_loganalysis_pipeline, stop_loganalysis_pipeline
from src.upload_model_pipeline import upload_model_pipeline, download_model_pipeline
from src.viewer_pipeline import start_viewer_pipeline, stop_viewer_pipeline

from src.types.hyperparameters import HyperParameters
from src.types.model_metadata import ModelMetadata
from src.config import settings

Using existing MinIO bucket: tcc-experiments
Using existing MinIO bucket: tcc-experiments
Using existing MinIO bucket: tcc-experiments
Using existing MinIO bucket: tcc-experiments
Using existing MinIO bucket: tcc-experiments
Viewer pipeline logging configured to file: /tmp/viewer_pipeline.log


/home/insightlab/miniconda3/envs/drfc/lib/python3.11/site-packages/gloe/functional.py:177: RuntimeWarning: Only one parameter is allowed on Transformers. Function 'setup_evaluation_env' has the following signature: (_, model_name: str, clone: bool = False, quiet: bool = False). To pass a complex data, use a complex type like named tuples, typed dicts, dataclasses or anything else.
  warnings.warn(
/home/insightlab/miniconda3/envs/drfc/lib/python3.11/site-packages/gloe/functional.py:177: RuntimeWarning: Only one parameter is allowed on Transformers. Function 'setup_upload_env' has the following signature: (_, source_model_name: str, target_model_name: Optional[str] = None, wipe: bool = False, force: bool = False). To pass a complex data, use a complex type like named tuples, typed dicts, dataclasses or anything else.
  warnings.warn(


## 2. Model Configuration

In [3]:
# Define a unique model name
model_name = 'rl-deepracer-demo'

# Create default hyperparameters and model metadata
hyperparameters = HyperParameters()
model_metadata = ModelMetadata()

In [4]:
# Display the hyperparameters
hyperparameters

HyperParameters(batch_size=64, beta_entropy=0.01, discount_factor=0.999, e_greedy_value=0.05, epsilon_steps=10000, exploration_type=<ExplorationType.CATEGORICAL: 'categorical'>, loss_type=<LossType.HUBER: 'huber'>, lr=0.0003, num_episodes_between_training=40, num_epochs=3, stack_size=1, term_cond_avg_score=100000, term_cond_max_episodes=100000)

In [5]:
# Display the model metadata
model_metadata

ModelMetadata(action_space_type=<ActionSpaceType.CONTINUOUS: 'continuous'>, action_space=ContinuousActionSpace(steering_angle=SteeringAngle(high=30.0, low=-30.0), speed=Speed(high=4.0, low=1.0)), version=5, training_algorithm=<TrainingAlgorithm.PPO: 'clipped_ppo'>, neural_network=<NeuralNetwork.DEEP_CONVOLUTIONAL_NETWORK_SHALLOW: 'DEEP_CONVOLUTIONAL_NETWORK_SHALLOW'>, sensor=[<Sensor.FRONT_FACING_CAMERA: 'FRONT_FACING_CAMERA'>])

In [6]:
def reward_function(params):
    """
    Example of a reward function for DeepRacer.
    
    Args:
        params (dict): Input parameters from the simulator
        
    Returns:
        float: The reward value
    """
    # Give a high reward by default
    reward = 1.0
    
    # Get track parameters
    all_wheels_on_track = params.get('all_wheels_on_track', True)
    distance_from_center = params.get('distance_from_center', 0)
    track_width = params.get('track_width', 1)
    
    # Calculate 3 markers that are at varying distances from the center line
    marker_1 = 0.1 * track_width
    marker_2 = 0.25 * track_width
    marker_3 = 0.5 * track_width
    
    # Give higher reward if the car is closer to center line and vice versa
    if distance_from_center <= marker_1:
        reward = 1.0
    elif distance_from_center <= marker_2:
        reward = 0.5
    elif distance_from_center <= marker_3:
        reward = 0.1
    else:
        reward = 1e-3  # likely crashed/ close to off track
        
    # Penalize if the car goes off track
    if not all_wheels_on_track:
        reward = 1e-3
        
    return float(reward)

## 3. Pipeline Operations

### 3.1 Training Pipeline

In [7]:
# First stop any existing training
stop_training_pipeline()

Attempting to stop training stack...
Targeting Run ID: 0
Cleaning up previous run for project deepracer-0...
Executing: docker compose -p deepracer-0 down --remove-orphans --volumes
Stderr:
 Container deepracer-0-rl_coach-1  Stopping
 Container deepracer-0-robomaker-1  Stopping
 Container deepracer-0-robomaker-1  Stopped
 Container deepracer-0-robomaker-1  Removing
 Container deepracer-0-robomaker-1  Removed
 Container deepracer-0-redis-1  Stopping
 Container deepracer-0-redis-1  Stopped
 Container deepracer-0-redis-1  Removing
 Container deepracer-0-redis-1  Removed
 Container deepracer-0-rl_coach-1  Stopped
 Container deepracer-0-rl_coach-1  Removing
 Container deepracer-0-rl_coach-1  Removed

Training stack stopped successfully.


In [8]:
# Start training with our model configuration
train_pipeline(
    model_name=model_name,
    hyperparameters=hyperparameters,
    model_metadata=model_metadata,
    reward_function=reward_function,
    overwrite=True,
    check_logs_after_start=False
)

Starting training pipeline for model: rl-deepracer-demo, Run ID: 0
Model prefix rl-deepracer-demo/ does not exist. Proceeding.
Successfully uploaded custom_files/hyperparameters.json to bucket tcc-experiments
Successfully uploaded custom_files/model_metadata.json to bucket tcc-experiments
Successfully uploaded custom_files/reward_function.py to bucket tcc-experiments
Data uploaded successfully to custom files
Successfully copied custom_files/reward_function.py to rl-deepracer-demo/reward_function.py
The reward function copied successfully to models folder at rl-deepracer-demo/reward_function.py
Generating local training_params.yaml...
Generated /tmp/dr/training-params-20250427220428.yaml, uploading to rl-deepracer-demo/training_params.yaml
Successfully uploaded local file /tmp/dr/training-params-20250427220428.yaml to rl-deepracer-demo/training_params.yaml
Verified: Training params file exists at rl-deepracer-demo/training_params.yaml
Cleaned up local file: /tmp/dr/training-params-2025

### 3.2 Viewer Pipeline

In [3]:
result = start_viewer_pipeline(delay=0)
print(f"View the training process at: {result['viewer_url']}")

View the training process at: http://localhost:8101


In [5]:
stop_viewer_pipeline()

{'status': 'success', 'message': 'Viewer and proxy processes stopped.'}

### 3.3 Log Analysis Pipeline

In [2]:
# Start log analysis container
result = start_loganalysis_pipeline()
pprint(result)

# You can now access the Jupyter notebook at http://localhost:8888

Starting log analysis container...


TypeError: Transformer.__call__() missing 1 required positional argument: 'data'

### 3.4 Upload/Download Pipeline

Let's assume we want to save our model under a different name to preserve it.

In [ ]:
# First let's stop the training to ensure all model files are properly saved
stop_training_pipeline()

In [ ]:
# Upload (or copy) the model to a new location
target_name = f"{model_name}-backup"
result = upload_model_pipeline(
    source_model_name=model_name,
    target_model_name=target_name,
    wipe=True,
    force=True
)
pprint(result)

In [ ]:
# We can also download a model (in this case, our backup to a new name)
result = download_model_pipeline(
    source_model_name=target_name,
    target_model_name=f"{model_name}-restored",
    force=True
)
pprint(result)

### 3.5 Evaluation Pipeline

Now let's evaluate the model to see how well it performs.

In [ ]:
# Stop any existing evaluation
stop_evaluation_pipeline()

In [ ]:
# Evaluate our trained model
result = evaluate_pipeline(
    model_name=model_name,
    quiet=False,
    clone=False
)
pprint(result)

# The evaluation is now running

In [ ]:
# Let's update the viewer to see the evaluation
result = start_viewer_pipeline(update=True)
pprint(result)

## 4. Cleanup

When you're done, it's good practice to stop all services to free up resources.

In [ ]:
# Stop all running services
stop_training_pipeline()
stop_evaluation_pipeline()
stop_viewer_pipeline()
stop_loganalysis_pipeline()

print("All DeepRacer services have been stopped.")